In [1]:
import pandas as pd

mean_diff = pd.read_csv("data/clean/influenced_names_means_diff.csv")
influenced_meandiff = mean_diff[mean_diff["Influence"] > 0]
# removing common identification mistakes such as "the", "a" or "Mr"
influenced_meandiff = influenced_meandiff[~influenced_meandiff["Character Name"].isin(["the", "a", "Mr"])]
print("Number of influenced names with mean diff: ", len(influenced_meandiff))
influenced_meandiff.sample()



Number of influenced names with mean diff:  1585


,Wikipedia ID,Movie Name,Year,Character Name,Count,Full name,Normalized_name,Influence
531,30006,the silence of the lambs,1991,Martin,2,Catherine Martin,MARTIN,418.6


In [2]:
prophet = pd.read_csv("data/clean/influenced_names_prophet.csv")
influenced_prophet = prophet[prophet["Influenced"] > 0]
# removing common identification mistakes such as "the", "a" or "Mr"
influenced_prophet = influenced_prophet[~influenced_prophet["Character Name"].isin(["the", "a", "Mr"])]
print("Number of influenced names with prophet: ", len(influenced_prophet))
influenced_prophet.head()

Number of influenced names with prophet:  432


,Wikipedia ID,Movie Name,Year,Character Name,Count,Full name,Normalized_name,Mean Difference,Influenced
0,451866,mission: impossible ii,2000,Ethan,18,Ethan Hunt,ETHAN,15725.466667,1
1,633411,the avengers,1998,Emma,15,Emma Peel,EMMA,14985.966667,1
2,3727473,man on fire,1987,Samantha,4,"Samantha ""Sam"" Balletto",SAMANTHA,14453.500000,1
3,347000,suspiria,1977,Sarah,15,Sarah,SARAH,14372.466667,1
5,320401,barton fink,1991,Taylor,3,Audrey Taylor,TAYLOR,13892.100000,1


In [25]:
import plotly.express as px
import plotly.graph_objects as go
names_per_year = influenced_prophet.groupby("Year").size().reset_index(name="Count")

# Create the figure
fig = go.Figure()

    # Add line for baby name count over time
fig.add_trace(
    go.Scatter(
        x=names_per_year['Year'], 
        y=names_per_year['Count'], 
        mode='lines',
        name='Baby Name Count',
        line=dict(color='blue')
    )
)

fig.update_layout(
        title="Number of Names influenced by movies per Year",
        xaxis_title='Year',
        yaxis_title='Count',
        legend_title="Legend",
        template="plotly_white",
        width=800
        )   

fig.show()
fig.write_html("docs/_includes/influenced_names_per_year.html")





In [16]:
import plotly.express as px
import plotly.graph_objects as go

sort_significant = influenced_prophet.sort_values(by='Year')


sort_significant['Label'] = sort_significant['Character Name'] + " from " + sort_significant['Movie Name']

# 2. Prepare list of names for each year
names_by_year = (
    sort_significant.groupby('Year')['Label']
    .apply(list)
    .to_dict()
)

# 3. Create a list of frames for each year
frames = []
for year, names in names_by_year.items():
    text = f"Influenced Names in {year}:<br>" + ",<br>".join(names)
    frame = go.Frame(
        data=[],
        name=str(year),
        layout=go.Layout(
            annotations=[
                dict(
                    text=text,
                    x=0.5, y=0.5,
                    xref="paper", yref="paper",
                    showarrow=False,
                    font=dict(size=16),
                    align="center",
                )
            ]
        )
    )
    frames.append(frame)

# 4. Create the initial figure
initial_year = min(names_by_year.keys())
initial_text = f"Influenced Names in {initial_year}:<br>" + ",<br>".join(names_by_year[initial_year])

fig = go.Figure(
    data=[],
    layout=go.Layout(
        annotations=[
            dict(
                text=initial_text,
                x=0.5, y=0.5,
                xref="paper", yref="paper",
                showarrow=False,
                font=dict(size=16),
                align="center",
            )
        ],
        sliders=[dict(
            active=0,
            currentvalue={"prefix": "Year: "},
            pad={"t": 50},
            steps=[
                dict(
                    method="animate",
                    args=[
                        [str(year)],  # Name of the frame
                        {"mode": "immediate", "frame": {"duration": 0, "redraw": True}},
                    ],
                    label=str(year),
                )
                for year in names_by_year.keys()
            ],
        )],
    ),
    frames=frames
)

# 5. Show the figure
fig.show()

